In [1]:
import os
import pandas as pd
import numpy as np

In [77]:
files = os.listdir(r'D:\My Documents\Downloads\kmrl_data\2017')
os.chdir(r'D:\My Documents\Downloads\kmrl_data\2017')

In [78]:
files_xlxs = [f for f in files if f[-4:] == 'xlsx' and f[0]!='~']
df = pd.DataFrame()

In [79]:
for f in files_xlxs:
    data = pd.read_excel(f)
    df = df.append(data)

In [91]:
df

,Station,Equipment Type,Equipment ID,Fare Media,Fare Product,Ticket/Card Number,Transaction Type,Date,Transaction Time
0,Pulinchodu,TOM,TOM 01,QR,SJT,B98783F4DD6821468C5B77C6AEBD2E3B,Issue,2017-07-01,2017-07-01 05:49:54.053
1,Pulinchodu,TOM,TOM 01,QR,SJT,576C696B01D774498F67B93A5FE43680,Issue,2017-07-01,2017-07-01 05:49:54.053
2,Pulinchodu,TOM,TOM 01,QR,SJT,A52AAA5E0AA4264C995FDBD3C3518B17,Issue,2017-07-01,2017-07-01 05:49:54.053
3,Palarivatom,TOM,TOM 01,QR,SJT,00C4290DC449334FB60CE0FB87E14E8E,Issue,2017-07-01,2017-07-01 05:50:03.647
4,Palarivatom,Gate,AG1113,QR,SJT,00C4290DC449334FB60CE0FB87E14E8E,Entry,2017-07-01,2017-07-01 05:50:30.000
5,Pulinchodu,Gate,AG204,QR,SJT,A52AAA5E0AA4264C995FDBD3C3518B17,Entry,2017-07-01,2017-07-01 05:50:31.000
6,Pulinchodu,Gate,AG204,QR,SJT,576C696B01D774498F67B93A5FE43680,Entry,2017-07-01,2017-07-01 05:50:38.000
7,Pulinchodu,Gate,AG204,QR,SJT,B98783F4DD6821468C5B77C6AEBD2E3B,Entry,2017-07-01,2017-07-01 05:50:42.000
8,Palarivatom,TOM,TOM 11,QR,SJT,7609411056059E4C98545BD275BAB148,Issue,2017-07-01,2017-07-01 05:52:12.777
9,Palarivatom,TOM,TOM 11,QR,SJT,6774362462B64549B5F3FC6DBB670039,Issue,2017-07-01,2017-07-01 05:52:12.777


In [7]:
store = pd.HDFStore('store.h5')
df = pd.DataFrame()
df = store['df'] 

In [43]:
store = pd.HDFStore('store.h5')

store['df'] = df  # save it
#store['df'] load it

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3325: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['Station', 'Equipment Type', 'Equipment ID', 'Fare Media', 'Fare Product', 'Ticket/Card Number', 'Transaction Type']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [272]:
df[(df['Fare Product'] == 'E-Purse')&(df['Transaction Type']=='Entry')].shape

(3548, 9)

In [273]:
df[(df['Fare Product'] == 'E-Purse')&(df['Transaction Type']=='Exit')].shape

(3423, 9)

In [148]:
import itertools
import datetime
from datetime import datetime,timedelta
start_date = datetime.strptime('2017-12-25',"%Y-%m-%d")
#end_date = start_date + 5*day_delta
#x = pd.DataFrame()
y = pd.DataFrame()

for i in range(0,5):
    kmon = df[(df['Date'] == start_date)]
    kma = kmon[(kmon['Transaction Type']=='Entry')&(((kmon['Fare Product']) == 'RJT')|(kmon['Fare Product'] == 'SJT'))]
    kmb = kmon[(kmon['Transaction Type']=='Exit')&(((kmon['Fare Product']) == 'RJT')|(kmon['Fare Product'] == 'SJT'))]
    kma = kma.reset_index(drop = True)
    kmb = kmb.reset_index(drop = True)
    kma = kma.drop(['Equipment Type','Equipment ID','Fare Media','Fare Product','Transaction Type','Date','Transaction Time'],axis =1)
    kmb = kmb.drop(['Equipment Type','Equipment ID','Fare Media','Fare Product','Transaction Type','Date','Transaction Time'],axis =1)
    kma.rename(index = str,columns = {"Ticket/Card Number": "ID"},inplace = True)
    kmb.rename(index = str,columns = {"Ticket/Card Number": "ID"},inplace = True)
    kma = kma.drop_duplicates(subset=['ID'])
    kmb = kmb.drop_duplicates(subset=['ID'])
    kmc = pd.merge(kma,kmb,on = 'ID', how = 'inner')
    date1 = start_date.strftime('%Y/%m/%d')
    kmc = kmc.groupby(['Station_x','Station_y']).size().reset_index(name = date1)
    a = kmc.loc[kmc['Station_x'].isin(['Edapally','Pathadipalam','Changampuzha Park']) & kmc['Station_y'].isin(['Maharajas College','M.G Road','Lissie'])]
    #a = kmc
    a = a.reset_index()
    y = pd.concat([x,a[date1]],axis=1)
    y[date1] = y[date1].fillna(0)
    x = y
    start_date = start_date + timedelta(days=1)

In [150]:
data_store = pd.HDFStore('2017_new.h5')

# Put DataFrame into the object setting the key as 'preprocessed_df'
data_store['pre_y'] = y
data_store.close()

In [153]:
x
    

,2017/10/03,2017/10/04,2017/10/05,2017/10/06,2017/10/09,2017/10/10,2017/10/11,2017/10/12,2017/10/13,2017/10/16,...,2017/12/18,2017/12/19,2017/12/20,2017/12/21,2017/12/22,2017/12/25,2017/12/26,2017/12/27,2017/12/28,2017/12/29
0,20,30,32,49,29,43,50,40,41,60,...,0.0,25,28,39,47,22,36,57,40,53
1,42,105,87,107,81,103,108,65,91,102,...,0.0,95,96,115,148,62,128,121,165,152
2,154,283,258,274,184,146,286,209,180,216,...,0.0,232,214,234,239,238,301,284,279,326
3,189,289,336,308,257,312,350,301,307,284,...,0.0,158,216,233,335,624,525,470,490,751
4,361,529,497,523,411,449,467,402,405,437,...,0.0,393,417,488,619,969,1087,1009,990,1091
5,794,1146,1126,1212,989,1003,1144,919,958,874,...,0.0,679,926,937,1279,2979,2917,2593,2618,2570
6,27,27,44,37,40,36,48,23,41,39,...,0.0,23,54,40,62,13,35,25,55,42
7,30,70,64,61,68,78,84,88,56,40,...,0.0,84,86,97,138,39,119,89,90,98
8,81,134,140,103,110,116,120,98,122,118,...,0.0,77,92,125,214,232,141,126,156,201


In [126]:
data_store['pre_y'] = y
data_store.close()

In [123]:
y = pd.concat([x,a['Size']],axis =1)
y['Size'] = y['Size'].fillna(0)
#a = y.iloc[5,-1]
#j = x.stack()
#j = np.array(j)
#j = j.reshape(1,-1)
#j = np.append(j,a)
#add = pd.DataFrame(j)
#list2 = list2.append(add.T,ignore_index = True)
#y = y.drop('Size_1',axis = 1)
x = y

In [532]:
store2['y']

,2018/01/01,2018/01/02,2018/01/03,2018/01/04,2018/01/05,2018/01/08,2018/01/09,2018/01/10,2018/01/11,2018/01/12,...,2018/02/15,2018/02/16,2018/02/19,2018/02/20,2018/02/21,2018/02/22,2018/02/23,2018/02/26,2018/02/27,2018/02/28
0,3.0,5,3,5,3.0,4.0,3.0,4,6,6,...,4.0,4,13,3,1,5,2.0,6.0,2,2.0
1,29.0,33,19,18,24.0,23.0,20.0,19,22,14,...,28.0,79,95,84,18,25,20.0,19.0,17,33.0
2,1.0,3,1,2,3.0,5.0,2.0,4,6,2,...,2.0,8,4,1,3,3,11.0,5.0,4,16.0
3,71.0,35,16,13,40.0,14.0,24.0,14,25,31,...,19.0,33,26,17,21,19,78.0,13.0,7,97.0
4,306.0,239,134,134,217.0,159.0,147.0,143,155,188,...,111.0,194,145,128,63,83,32.0,105.0,72,39.0
5,126.0,115,75,71,79.0,76.0,79.0,43,89,43,...,80.0,83,52,45,45,25,5.0,25.0,52,3.0
6,1.0,1,4,2,4.0,13.0,2.0,2,8,2,...,5.0,9,8,9,4,1,14.0,7.0,2,3.0
7,18.0,9,13,13,12.0,16.0,8.0,7,7,3,...,0.0,23,32,39,13,9,4.0,1.0,14,4.0
8,0.0,10,2,4,0.0,0.0,0.0,6,1,3,...,0.0,5,6,6,5,1,0.0,0.0,2,0.0


In [124]:
y

,index,Station_x,Station_y,Size,Size,Size
0,31,Changampuzha Park,Ambattukavu,11.0,7.0,8.0
1,37,Changampuzha Park,Kalamassery,36.0,14.0,37.0
2,42,Changampuzha Park,Muttom,2.0,2.0,11.0
3,78,Edapally,Ambattukavu,33.0,60.0,42.0
4,84,Edapally,Kalamassery,300.0,319.0,207.0
5,89,Edapally,Muttom,86.0,138.0,56.0
6,221,Pathadipalam,Ambattukavu,7.0,4.0,9.0
7,227,Pathadipalam,Kalamassery,7.0,2.0,8.0
8,232,Pathadipalam,Muttom,3.0,0.0,6.0


In [125]:
kmc

,Station_x,Station_y,Size
0,Aluva,Aluva,4
1,Aluva,Ambattukavu,43
2,Aluva,Changampuzha Park,127
3,Aluva,Cochin University,139
4,Aluva,Companypady,30
5,Aluva,Edapally,996
6,Aluva,JLN Stadium,144
7,Aluva,Kalamassery,146
8,Aluva,Kaloor,337
9,Aluva,Lissie,408


In [17]:
import h5py

filename = "store1.h5"

h5 = h5py.File(filename,'r')

OSError: Unable to open file (bad object header version number)

In [57]:
x = pd.DataFrame()

In [51]:
store = pd.io.pytables.HDFStore('store.h5')


In [60]:
store.close()

HDF5ExtError: HDF5 error back trace

  File "C:\ci\hdf5_1545244154871\work\src\H5G.c", line 682, in H5Gclose
    not a group

End of HDF5 error back trace

Problems closing the Group y

In [61]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: store.h5

In [62]:
store[]

SyntaxError: invalid syntax (<ipython-input-62-ccf368325223>, line 1)

In [63]:
data_store = pd.HDFStore('store.h5')

In [64]:
preprocessed_df = data_store['df']

KeyError: 'No object named df in the file'

In [69]:
df

,Station,Equipment Type,Equipment ID,Fare Media,Fare Product,Ticket/Card Number,Transaction Type,Date,Transaction Time
0,Pulinchodu,TOM,TOM 01,QR,SJT,B98783F4DD6821468C5B77C6AEBD2E3B,Issue,2017-07-01,2017-07-01 05:49:54.053
1,Pulinchodu,TOM,TOM 01,QR,SJT,576C696B01D774498F67B93A5FE43680,Issue,2017-07-01,2017-07-01 05:49:54.053
2,Pulinchodu,TOM,TOM 01,QR,SJT,A52AAA5E0AA4264C995FDBD3C3518B17,Issue,2017-07-01,2017-07-01 05:49:54.053
3,Palarivatom,TOM,TOM 01,QR,SJT,00C4290DC449334FB60CE0FB87E14E8E,Issue,2017-07-01,2017-07-01 05:50:03.647
4,Palarivatom,Gate,AG1113,QR,SJT,00C4290DC449334FB60CE0FB87E14E8E,Entry,2017-07-01,2017-07-01 05:50:30.000
5,Pulinchodu,Gate,AG204,QR,SJT,A52AAA5E0AA4264C995FDBD3C3518B17,Entry,2017-07-01,2017-07-01 05:50:31.000
6,Pulinchodu,Gate,AG204,QR,SJT,576C696B01D774498F67B93A5FE43680,Entry,2017-07-01,2017-07-01 05:50:38.000
7,Pulinchodu,Gate,AG204,QR,SJT,B98783F4DD6821468C5B77C6AEBD2E3B,Entry,2017-07-01,2017-07-01 05:50:42.000
8,Palarivatom,TOM,TOM 11,QR,SJT,7609411056059E4C98545BD275BAB148,Issue,2017-07-01,2017-07-01 05:52:12.777
9,Palarivatom,TOM,TOM 11,QR,SJT,6774362462B64549B5F3FC6DBB670039,Issue,2017-07-01,2017-07-01 05:52:12.777


In [127]:
y

,2017/06/19,2017/06/20,2017/06/21,2017/06/24,2017/06/25,2017/06/26,2017/06/27,2017/06/28,2017/07/01,2017/07/02,...,2017/12/18,2017/12/19,2017/12/20,2017/12/23,2017/12/24,2017/12/25,2017/12/26,2017/12/27,2017/12/30,2017/12/31
0,65,61,27,28,94,71,20,13,16,48,...,0.0,2,1.0,2,5.0,3.0,3.0,5,4,2
1,485,167,180,200,351,294,93,156,175,254,...,0.0,18,17.0,25,11.0,16.0,38.0,20,39,38
2,108,50,23,38,41,81,16,17,7,77,...,0.0,3,4.0,1,4.0,41.0,87.0,1,11,9
3,173,109,76,189,166,354,286,228,169,223,...,0.0,15,13.0,24,47.0,317.0,310.0,37,71,88
4,717,726,589,634,890,988,750,790,617,1125,...,0.0,91,90.0,166,343.0,160.0,119.0,325,375,440
5,313,215,195,256,417,520,508,337,301,466,...,0.0,29,25.0,103,124.0,5.0,12.0,170,85,88
6,34,45,29,17,42,48,20,23,13,20,...,0.0,2,1.0,54,1.0,19.0,12.0,13,7,9
7,101,69,28,45,70,39,43,34,25,50,...,0.0,4,4.0,5,13.0,22.0,7.0,16,7,11
8,52,33,21,11,20,38,47,26,6,39,...,0.0,4,0.0,5,0.0,0.0,0.0,2,5,12


In [128]:
data_store = pd.HDFStore('2017.h5')

In [129]:
data_store.close()

In [7]:
data_store = pd.HDFStore('2017_1.h5')

# Retrieve data using key
preprocessed_df = data_store['pre_y']
data_store.close()

In [8]:
preprocessed_df

,2017/06/19,2017/06/20,2017/06/21,2017/06/24,2017/06/25,2017/06/26,2017/06/27,2017/06/28,2017/07/01,2017/07/02,...,2017/12/18,2017/12/19,2017/12/20,2017/12/23,2017/12/24,2017/12/25,2017/12/26,2017/12/27,2017/12/30,2017/12/31
0,65,61,27,28,94,71,20,13,16,48,...,0.0,2,1.0,2,5.0,3.0,3.0,5,4,2
1,485,167,180,200,351,294,93,156,175,254,...,0.0,18,17.0,25,11.0,16.0,38.0,20,39,38
2,108,50,23,38,41,81,16,17,7,77,...,0.0,3,4.0,1,4.0,41.0,87.0,1,11,9
3,173,109,76,189,166,354,286,228,169,223,...,0.0,15,13.0,24,47.0,317.0,310.0,37,71,88
4,717,726,589,634,890,988,750,790,617,1125,...,0.0,91,90.0,166,343.0,160.0,119.0,325,375,440
5,313,215,195,256,417,520,508,337,301,466,...,0.0,29,25.0,103,124.0,5.0,12.0,170,85,88
6,34,45,29,17,42,48,20,23,13,20,...,0.0,2,1.0,54,1.0,19.0,12.0,13,7,9
7,101,69,28,45,70,39,43,34,25,50,...,0.0,4,4.0,5,13.0,22.0,7.0,16,7,11
8,52,33,21,11,20,38,47,26,6,39,...,0.0,4,0.0,5,0.0,0.0,0.0,2,5,12


In [1]:
pwd

'C:\\Users\\HP ProBook'